### Title: Automated Face Blurring using MTCNN 

**(Facenet-Pytorch)**

### Step-by-Step Description

1. **Install dependencies**

   * Install required Python packages:

     ```bash
     !pip install facenet-pytorch tqdm moviepy
     ```
   * These packages support face detection (`facenet-pytorch`), progress visualization (`tqdm`), and video processing (`moviepy`).

2. **Verify PyTorch and GPU setup**

   * Prints the PyTorch version.
   * Checks if GPU (`cuda`) is available for faster processing.

3. **Import necessary libraries**

   * `cv2` for video I/O.
   * `torch` for GPU operations.
   * `time` to calculate processing time.
   * `tqdm` for progress bars.
   * `MTCNN` from `facenet_pytorch` for face detection.
   * `PIL.Image` for image conversions.
   * `numpy` for array manipulation.

4. **Initialize MTCNN**

   * Loads the MTCNN face detector with `keep_all=True` to detect multiple faces in each frame.
   * Selects GPU if available, otherwise defaults to CPU.

5. **Set input and output paths**

   * Input path: `/kaggle/input/bd-sports-10/Dataset/Lathim` (can be changed to any dataset folder/class).
   * Output path: `/kaggle/working/Lathim_blurred` where blurred videos are saved.

6. **List all video files**

   * Collects all `.mp4` files from the input folder.

7. **Process videos with progress tracking**

   * Iterates over all videos using `tqdm` (green progress bar).
   * For each video:

     * Opens with `cv2.VideoCapture`.
     * Reads metadata (FPS, width, height, total frames).
     * Initializes `cv2.VideoWriter` to save the processed video.

8. **Frame-by-frame processing**

   * For each frame:

     * Converts the frame from BGR → RGB (OpenCV → PIL).
     * Detects face bounding boxes with MTCNN.
     * For each detected face, extracts the region and applies **Gaussian blur** (`cv2.GaussianBlur`).
     * Replaces the original face region with the blurred version.

9. **Save processed video**

   * Each blurred frame is written back into the output video file.
   * Once finished, the video is released and saved.
   * Processing time per video is displayed.

10. **Finalize output**

    * After all videos are processed, the script prints a success message.

11. **Zip the output folder**

    * The blurred videos folder (`/kaggle/working/Lathim_blurred`) is compressed into a `.zip` file for convenient download and sharing.

---

👉 Just like your resizing code, the **input/output paths can be changed** to handle other sports categories.
For example:

```python
input_path = "/kaggle/input/bd-sports-10/Dataset/Kabaddi"
output_path = "/kaggle/working/Kabaddi_blurred"
```

---




In [ ]:
!pip install facenet-pytorch tqdm moviepy


In [ ]:
import torch
print(torch.__version__)  # Verify the PyTorch version
print(torch.cuda.is_available())  # Check if GPU is available


# MTCNN Implementation 

In [ ]:
import os
import cv2
import torch
import time
from tqdm import tqdm
from facenet_pytorch import MTCNN
from PIL import Image
import numpy as np

# Initialize MTCNN for face detection
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=True, device=device)

# Input and output paths
input_path = '/kaggle/input/bd-sports-10/Dataset/Lathim'
output_path = '/kaggle/working/Lathim_blurred'
os.makedirs(output_path, exist_ok=True)

# List of video files
video_files = [f for f in os.listdir(input_path) if f.endswith('.mp4')]

# Loop over video files with tqdm (one progress bar per video)
for filename in tqdm(video_files, desc="Processing videos"):
    start_time = time.time()

    video_path = os.path.join(input_path, filename)
    output_video_path = os.path.join(output_path, f'blurred_{filename}')

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for _ in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR (OpenCV) to RGB (PIL)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_frame)

        # Detect faces
        boxes, _ = mtcnn.detect(pil_img)

        # Blur each face
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)

                face_region = frame[y1:y2, x1:x2]
                blurred_face = cv2.GaussianBlur(face_region, (99, 99), 30)
                frame[y1:y2, x1:x2] = blurred_face

        out.write(frame)

    cap.release()
    out.release()
    elapsed = time.time() - start_time
    tqdm.write(f"✅ Finished: {filename} | Time taken: {elapsed:.2f} sec")

print("🎉 All videos processed.")


# Converting to Zip file

In [ ]:
import shutil

# Path to folder you want to zip
folder_to_zip = '/kaggle/working/Lathim_blurred'
zip_path = '/kaggle/working/Lathim_blurred.zip'

# Create a zip file
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_to_zip)
